## 1. Setup & Installation

In [ ]:
# Check CUDA availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Install dependencies (if needed)
# !pip install -r ../requirements.txt

## 2. Prepare Training Dataset

In [ ]:
import sys
sys.path.append('..')

from app.prepare_dataset import prepare_dataset, validate_dataset

# Create sample dataset
samples = prepare_dataset(
    input_dir="../data/processed",
    output_file="../data/ielts_training_data.jsonl"
)

print(f"\nCreated {len(samples)} training samples")

In [ ]:
# Validate dataset
validate_dataset("../data/ielts_training_data.jsonl")

## 3. Initialize Trainer

In [ ]:
from app.train_qlora import IELTSModelTrainer

# Initialize trainer
trainer = IELTSModelTrainer(
    model_name="qwen",  # Options: qwen, phi2, gemma
    output_dir="../models/ielts-demo",
    dataset_path="../data/ielts_training_data.jsonl"
)

print("✓ Trainer initialized")

## 4. Train Model with QLoRA

**Note**: Training có thể mất 10-30 phút tùy vào dataset size

In [ ]:
# Train model
trained_model = trainer.train(
    num_epochs=3,
    batch_size=1,  # Small batch for 4GB GPU
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_seq_length=512,
)

print("\n✓ Training complete!")

## 5. Test Inference

In [ ]:
from app.inference import AutotrainInferenceServer
import json

# Load trained model
server = AutotrainInferenceServer(model_path="../models/ielts-demo")

print("✓ Model loaded for inference")

In [ ]:
# Test with sample transcript
test_transcript = """
Well, I think that technology has greatly influenced our daily lives. 
For example, smartphones allow us to stay connected with friends and family. 
Moreover, the internet provides us with access to vast amounts of information.
""".strip()

print("📝 Test Transcript:")
print(test_transcript)
print("\n🎯 Scoring...")

scores = server.score_transcript(test_transcript)
print(json.dumps(scores, indent=2))

## 6. Convert to GGUF (Optional)

Convert model to GGUF format for llama.cpp CPU inference

In [ ]:
from app.convert_model import convert_to_gguf

# Convert to GGUF
gguf_path = convert_to_gguf(
    model_path="../models/ielts-demo",
    output_path="../models/ielts-demo/model.gguf",
    quantization="Q4_K_M"  # 4-bit quantization
)

if gguf_path:
    print(f"✓ Model converted to {gguf_path}")

## 7. Benchmark Performance

In [ ]:
import time

# Benchmark inference speed
test_cases = [
    "I like technology.",
    "Well, I think education is important for personal development.",
    "In my opinion, environmental protection requires collective effort from all members of society."
]

print("📊 Benchmarking inference speed...\n")

for i, transcript in enumerate(test_cases, 1):
    start = time.time()
    scores = server.score_transcript(transcript)
    elapsed = time.time() - start
    
    print(f"Test {i}: {elapsed*1000:.2f}ms")
    print(f"  Transcript length: {len(transcript.split())} words")
    print()

## 8. Compare Different Models

In [ ]:
# Train and compare different base models
models_to_test = ["qwen", "phi2", "gemma"]

results = {}

for model_name in models_to_test:
    print(f"\n{'='*50}")
    print(f"Testing {model_name.upper()}")
    print('='*50)
    
    # Note: This will take a while!
    # trainer = IELTSModelTrainer(
    #     model_name=model_name,
    #     output_dir=f"../models/ielts-{model_name}",
    # )
    # trainer.train(num_epochs=1, batch_size=1)
    
    print(f"✓ {model_name} training complete")

## 9. Export for Production

Final checklist before deployment

In [ ]:
import os
from pathlib import Path

model_dir = Path("../models/ielts-demo")

# Check required files
required_files = [
    "config.json",
    "adapter_config.json",
    "adapter_model.safetensors",
    "tokenizer.json",
    "tokenizer_config.json",
]

print("📦 Deployment Checklist:\n")

for file in required_files:
    exists = (model_dir / file).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {file}")

print("\n✓ Model ready for deployment!")

## Next Steps

1. ✅ Model đã được train với QLoRA
2. ✅ Test inference thành công
3. 🔄 Deploy với vLLM hoặc llama.cpp
4. 🔄 Integrate vào production API
5. 🔄 Monitor performance và fine-tune thêm

Xem thêm:
- [Training Guide](../docs/TRAINING_DEPLOYMENT.md)
- [Quick Start](../docs/QUICKSTART.md)